<a href="https://colab.research.google.com/github/JayantJharkhande3000/CS6910_Assignment/blob/main/Part_A_Question_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
import pathlib
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from PIL import Image

In [43]:
data_directory = pathlib.Path('/content/drive/MyDrive/DATA/nature_12K/inaturalist_12K')
training_data = tf.keras.preprocessing.image_dataset_from_directory(
                      directory = "/content/drive/MyDrive/DATA/nature_12K/inaturalist_12K/train",
                      labels = 'inferred',  
                      label_mode = 'categorical',
                      color_mode = 'rgb',
                      batch_size = 32,
                      image_size = (256, 256),
                      shuffle = True,
                      seed = 17,
                      validation_split = 0.2,
                      subset = 'training')

validation_data = tf.keras.preprocessing.image_dataset_from_directory(
                      directory = "/content/drive/MyDrive/DATA/nature_12K/inaturalist_12K/val",
                      labels = 'inferred',  
                      label_mode = 'categorical',
                      color_mode = 'rgb',
                      batch_size = 32,
                      image_size = (256, 256),
                      shuffle = True,
                      seed = 17,
                      validation_split = 0.9,
                      subset = 'validation')

Found 9999 files belonging to 10 classes.
Using 8000 files for training.
Found 2000 files belonging to 10 classes.
Using 1800 files for validation.


In [44]:
first_layer_filters = np.array([32,32,32,32,32])
filter_org = 1
data_aug = False
batch_norm = False
dropout = 0.2
kernel = np.array([9, 7, 5, 3, 3])
dense_size = 32
activation = 'relu'
num_epochs = 3
optimizer = 'adam'
conv_layers = 5
print(first_layer_filters)
print(kernel)


[32 32 32 32 32]
[9 7 5 3 3]


In [45]:
inputs_Image = tf.keras.Input(shape = (256, 256, 3))
x_image_data = Rescaling(scale = 1.0/255)(inputs_Image)
filter_sizes = first_layer_filters

In [46]:
for layer_num in range(conv_layers):
  k  = kernel[layer_num]
  f = filter_sizes[layer_num]
  x_image_data = layers.Conv2D(filters = f, kernel_size = (int(k),int(k)))(x_image_data)
  if batch_norm:
    x_image_data = layers.BatchNormalization(axis = -1)(x_image_data)
  x_image_data = layers.Activation(activation)(x_image_data)
  x_image_data = layers.MaxPooling2D(pool_size = (2, 2))(x_image_data)  

In [47]:
x_image_data = layers.Flatten()(x_image_data)
x_image_data = layers.Dense(dense_size)(x_image_data)
if batch_norm:
    x_image_data = layers.BatchNormalization(axis = -1)(x_image_data)
x_image_data = layers.Activation(activation)(x_image_data)
if dropout > 0:
    x_image_data = layers.Dropout(rate = dropout)(x_image_data) 

In [48]:
outputs_layer = layers.Dense(10, activation ='softmax')(x_image_data)
model = tf.keras.Model(inputs = inputs_Image, outputs = outputs_layer)

In [49]:
model.compile(optimizer= optimizer, loss = tf.keras.losses.CategoricalCrossentropy(name = 'loss'),
                      metrics = [tf.keras.metrics.CategoricalAccuracy(name = 'acc')])

In [50]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 rescaling_6 (Rescaling)     (None, 256, 256, 3)       0         
                                                                 
 conv2d_12 (Conv2D)          (None, 248, 248, 32)      7808      
                                                                 
 activation_16 (Activation)  (None, 248, 248, 32)      0         
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 124, 124, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 118, 118, 32)      50208     
                                                           

In [51]:
model_history = model.fit(training_data, epochs = num_epochs, validation_data = validation_data)

Epoch 1/3
250/250 [==============================] - 2291s 9s/step - loss: 2.3032 - acc: 0.0951 - val_loss: 2.3025 - val_acc: 0.1039
Epoch 2/3
  5/250 [..............................] - ETA: 35:35 - loss: 2.3017 - acc: 0.1250

KeyboardInterrupt: ignored